In [ ]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import MultifieldParser, OrGroup

In [ ]:
schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT(stored=True))
schema

In [ ]:
ix = create_in("indexdir", schema)
writer = ix.writer()
writer.add_document(
    title=u"My document", content=u"This is my python document! hello big world",
)
writer.add_document(
    title=u"Second try", content=u"This is the second example hello world.",
)
writer.add_document(
    title=u"Third time's the charm", content=u"More examples. Examples are many.",
)
writer.commit()

In [ ]:
with ix.searcher() as searcher:
    parser = MultifieldParser(["many", "content"], ix.schema, group=OrGroup)
    query_content = ["charm", "world"]
    myquery = parser.parse(" ".join(query_content))
    corrector = searcher.corrector("content")
    for mistyped_word in query_content:
        print(corrector.suggest(mistyped_word, limit=1))
    
    results = searcher.search(myquery, terms=True)
    
    for result in results:
        print(result.get('title'), result.score)

In [ ]:
with ix.searcher() as searcher:
    parser = QueryParser("content", ix.schema)
    query_content = ["hello", "world"]
    myquery = parser.parse(" ".join(query_content))
    corrector = searcher.corrector("content")
    for mistyped_word in query_content:
        print(corrector.suggest(mistyped_word, limit=3))
    
    results = searcher.search(myquery, terms=True)
    
    print(results[0], results[0].score)